In [ ]:
import os
import cv2
import csv

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
num_output = 8
input_shape = (512, 512, 3)

In [ ]:
batch_size = 10

In [ ]:
IMAGES_FOLDER = '/home/sanjeev309/Projects/posebox/resized_frames'
ANNOTATION_FILE = '/home/sanjeev309/Projects/posebox/annotation_formatted.csv'
OUTPUT = '/home/sanjeev309/Projects/posebox/output'

In [ ]:
checkpoint_dir = OUTPUT + "/ckpt"

### Initialise empty numpy arrays

In [ ]:
data = np.empty((0,512,512,3), dtype=np.int8)
target = np.empty((0,8), dtype=np.float)

In [ ]:
data.shape

In [ ]:
target.shape

### Read annotation file, fetch image, normalise image and array, compose data and target arrays

In [ ]:
with open(ANNOTATION_FILE,'r') as csv_file:
    
    reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    
    for row in reader:
        
        print(row)
        
        if line_count == 0:
            line_count += 1
        else:
            image_path = os.path.join(IMAGES_FOLDER, row[0])
            image = cv2.imread(image_path)/ 255
            image = np.expand_dims(image, axis=0)
            
            points = row[1]
            dimen = (float)(row[2])
            
            p = points.strip('][').split(', ')
            
            
            p = np.array(p, dtype=np.int)
            p = np.divide(p, dimen)
            p = np.expand_dims(p, axis=0)
            
            if image is not None:
                data = np.vstack((data, image))
                target = np.vstack((target, p))
            
            line_count += 1

### Shuffle data and target synchronously

In [ ]:
num_samples = data.shape[0]

In [ ]:
arr = np.arange(num_samples)

In [ ]:
np.random.shuffle(arr)

In [ ]:
arr

In [ ]:
data = data[arr]
target = target[arr]

In [ ]:
print(data.shape)
print(target.shape)

In [ ]:
np.save(os.path.join(OUTPUT,'data.npy'), data)
np.save(os.path.join(OUTPUT,'target.npy'), target)

### Load data and target

In [ ]:
data = np.load(os.path.join(OUTPUT,'data.npy'))
target = np.load(os.path.join(OUTPUT,'target.npy'))

### Train / Test split

In [ ]:
TRAIN_RATIO = 0.8

In [ ]:
X_train = data[0: int(num_samples * TRAIN_RATIO) - 1]
y_train = target[0: int(num_samples * TRAIN_RATIO) - 1]

X_test = data[int(num_samples * TRAIN_RATIO): num_samples - 1]
y_test = target[int(num_samples * TRAIN_RATIO): num_samples - 1]

In [ ]:
print(X_train.shape)

In [ ]:
print(y_train.shape)

In [ ]:
print(X_test.shape)

In [ ]:
print(y_test.shape)

In [ ]:
def build_model():
    base_model = keras.applications.MobileNetV2(input_shape=(127, 127,3),
                                                include_top=False,
                                                weights='imagenet')

    base_model.trainable = False

    print(base_model.summary())

    model = keras.Sequential([
        
        layers.Input(shape=input_shape),
        layers.Conv2D(3, (2,2),activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Conv2D(3,(2,2), activation='relu'),
        layers.MaxPool2D((2,2)),
        base_model,
        layers.Flatten(),
        layers.Dense(64),
        layers.Dense(32),
        layers.Dense(16),
        layers.Dense(num_output, activation="sigmoid"),

    ])
    optimizer = keras.optimizers.SGD(0.01)

    model.compile(loss=keras.losses.mean_squared_error,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [ ]:
def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + '/' + name
                   for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print('Restoring from', latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print('Creating a new model')
    return build_model()

In [ ]:
model = make_or_restore_model()
callbacks = [
    # This callback saves a SavedModel every 5 epochs.
    # We include the training loss in the folder name.
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir + '/ckpt-loss={loss:.2f}',
        # save_freq=4)
        period=5)
]

In [ ]:
model.summary()

In [ ]:
# del model

In [ ]:
train_history = model.fit(data, target, batch_size=batch_size, validation_split= 0.2, epochs=5000,callbacks=callbacks)